In [384]:
%matplotlib inline
from __future__ import print_function

from matplotlib import pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2
 
from utils.nolearn_net import NeuralNet
from utils.data_utils import *
from utils.iterator import BaseBatchIterator
from utils.training_utils import parse_conf_file
from utils.preprocessing import build_pipeline
import importlib
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Test 

In [374]:
config = parse_conf_file('/Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001/conf_model.json')

In [375]:
config

{u'architecture': u'build_simple_lstm',
 u'build_ite': u'benchmark',
 u'conf': u'../models/lstm/model_3/conf_model.json',
 u'continue_training': False,
 u'feats': [u'percip_mm',
  u'humidity',
  u'temp',
  u'leafwet450_min',
  u'leafwet460_min',
  u'leafwet_lwscnt',
  u'gusts_ms',
  u'wind_dir',
  u'wind_ms'],
 u'folder': u'/Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001',
 u'grad_clip': 50,
 u'hiddens': 25,
 u'loss_function': u'squared_error',
 u'lr': 0.001,
 u'model': u'lstm',
 u'model_fname': u'/Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001/model.pkl',
 u'model_graph_fname': u'/Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001/model_history.png',
 u'model_history_fname': u'/Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001/model_history.pkl',
 u'model_name': u'h_25_rg_1e-07_lr_0.001',
 u'name': u'micro',
 u

In [381]:
pipeline = build_pipeline(config['pipe_list'], config['pipe_kwargs'])
nb_features, batch_ite_train, batch_ite_val, batch_ite_test, batch_ite_pred = load_data(
        name=config['name'],
        feats=config['feats'],
        build_ite=config['build_ite'],
        pipeline=pipeline)
model = importlib.import_module(
        'model_defs.%s' % config['model'])
builder = getattr(model, config['architecture'])
architecture = builder(D=nb_features, H=config[
        'hiddens'], grad_clip=config['grad_clip'])

Le train is composed by 21 group and 3738 observation
Le val is composed by 8 group and 1112 observation
Le test is composed by 5 group and 948 observation


In [385]:
net = NeuralNet(
        layers=architecture,
        regression=True,
        objective_loss_function=getattr(
            lasagne.objectives, config['loss_function']),
        objective_l2=config['reg'],  # L2 regularization
        update=getattr(lasagne.updates, config['update_rule']),
        update_learning_rate=config['lr'],
        batch_iterator_train=batch_ite_train,
        batch_iterator_test=batch_ite_val,
        verbose=config['verbose'],
        max_epochs=10000,)
net.initialize()
net.load_params_from(config['model_fname'])
with open(config['model_history_fname']) as f:
    net.train_history_ = pickle.load(f)

Loaded parameters to layer 'lstm' (shape 9x25).
Loaded parameters to layer 'lstm' (shape 25x25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 9x25).
Loaded parameters to layer 'lstm' (shape 25x25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 9x25).
Loaded parameters to layer 'lstm' (shape 25x25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 9x25).
Loaded parameters to layer 'lstm' (shape 25x25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 1x25).
Loaded parameters to layer 'lstm' (shape 1x25).
Loaded parameters to layer 'dense' (shape 25x1).
Loaded parameters to layer 'dense' (shape 1).


In [388]:
net.get_score_whole_set('train'),net.get_score_whole_set('test')

(3.0808924682517103, 5.8183123682635864)

In [389]:
from utils.hook import write_final_score

In [390]:
write_final_score(config,net)

In [395]:
from utils.prediction_utils import *

In [397]:
    final_pred = prediction(net, batch_ite_pred)
    make_submission(config, final_pred)

Will write output to /Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001/submissions_20160322_144424.csv


In [398]:
>>> def producer():
...     for i in range(6):
...         print('Produced %s' % i)
...         yield i


In [399]:
>>> from math import sqrt
>>> from joblib import Parallel, delayed

In [400]:
out = Parallel(n_jobs=2, verbose=100, pre_dispatch='1.5*n_jobs')(
...                         delayed(sqrt)(i) for i in producer()) 

Produced 0
Produced 1
Produced 2
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Batch computation too fast (0.0057s.) Setting batch_size=70.
Produced 3
Produced 4
Produced 5
[Parallel(n_jobs=2)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   5 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   5 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   6 out of   6 | elapsed:    0.0s finished


In [409]:
import itertools
def run_generator(parameters_grids):

    assert all([f in ['lr', 'reg', 'hiddens'] for f in parameters_grid.keys()])
    product = [x for x in apply(itertools.product, parameters_grid.values())]
    runs = [dict(zip(parameters_grid.keys(), p)) for p in product]
    for run in runs:
        yield run

In [411]:
    parameters_grid = {'lr': np.logspace(-7, -2, num=100),
                       'reg': [1e-7],
                       'hiddens': [25, 50]}

In [416]:
import time

In [417]:
def test(config):
    time.sleep(0.2)
    return config['lr']

In [418]:
runs = run_generator(parameters_grid)

In [419]:
out = Parallel(n_jobs=2, verbose=100)(
...                         delayed(test)(conf) for conf in runs) 

[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done   3 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done   5 tasks      | elapsed:    0.6s
[Parallel(n_jobs=2)]: Done   6 tasks      | elapsed:    0.6s
[Parallel(n_jobs=2)]: Done   7 tasks      | elapsed:    0.8s
[Parallel(n_jobs=2)]: Done   8 tasks      | elapsed:    0.8s
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:    1.0s
[Parallel(n_jobs=2)]: Done  10 tasks      | elapsed:    1.0s
[Parallel(n_jobs=2)]: Done  11 tasks      | elapsed:    1.2s
[Parallel(n_jobs=2)]: Done  12 tasks      | elapsed:    1.2s
[Parallel(n_jobs=2)]: Done  13 tasks      | elapsed:    1.4s
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    1.4s
[Parallel(n_jobs=2)]: Done  15 tasks      | elapsed:    1.6s
[Parallel(n_jobs=2)]: Done  16 tasks      | elapsed:    1.6s
[Parallel(n_jobs=2)]: Do